In [3]:
import cv2
from pathlib import Path 
import sys
sys.path.append(str(Path.cwd().parent.parent))
from utils import create_fh_logger

In [ ]:
src = Path.cwd().parent.parent.parent / 'processing' / 'nro_declassified' / 'imgs'
pdfs = sorted(list(src.glob('*img')))
dst = src.parent / 'cleaned_imgs'
dst.mkdir(exist_ok=True)
logs = src.parent.parent / 'logs'
logs.mkdir(exist_ok=True)
logger = create_fh_logger(logs / "imgs2pdfs.log")